In [ ]:
from dask.distributed import Client, as_completed
import time
import random

In [ ]:
client = Client('localhost:8786', timeout=1000)
client

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/3/32/Rosenbrock_function.svg/300px-Rosenbrock_function.svg.png" align="right">

### Rosenbrock function

(sorry for the Jet colormap)

In [ ]:
def rosenbrock(point):
    """Compute the rosenbrock function and return the point and result"""
    time.sleep(0.1)
    score = (1 - point[0])**2 + 2 * (point[1] - point[0]**2)**2
    return point, score


scale = 5                  # Intial random perturbation scale
best_point = (1, 2)        # Best point so far
best_score = float('inf')  # Best score so far

# Initially just pick some random points to evaluate
initial = [(random.uniform(-5, -5), random.uniform(-5, 5))
           for i in range(10)]

In [ ]:
futures = [client.submit(rosenbrock, point) for point in initial]

running = as_completed(futures)

for res in running:
    point, score = res.result()
    if score < best_score:
        best_score, best_point = score, point
        print("Current best (%.3f, %.3f)."
              "Scale: %.3f" % (best_point + (scale,)))
    
    x, y = best_point
    new_point =  (x + random.uniform(-scale, scale),
                  y + random.uniform(-scale, scale))
    new_point = client.submit(rosenbrock, new_point)
                                           
    running.add(new_point)
    
    scale *= 0.99

    if scale < 0.001:
        break